# Reddit Comments from May 2015

This dataset contains every publicly available Reddit comment for the month of May, 2015. Approximately 37 million comments were made by 2.7 million unique authors, stored in a ~15 GB compressed file.

In [4]:
from spark.dataset import Dataset


hdfs_path = "hdfs://namenode:9000/user/spark/reddit_comments/"
spark_master = "spark://spark-master:7077"

dataset = Dataset(hdfs_path, spark_master)
dataset.pre_process()

# Test
dataset.dataframe.sort("score", ascending=False).show(10)
dataset.dataframe.describe().show()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/03 21:40:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+--------------------+-----+---------+
|                body|score|subreddit|
+--------------------+-----+---------+
|Then you got your...| 6761|AskReddit|
|Thanks man and al...| 5849|AskReddit|
|We had an office ...| 5776|AskReddit|
|OP are you alrigh...| 5767|AskReddit|
|So she's pregnant...| 5762|AskReddit|
|Forum based websi...| 5710|AskReddit|
|It would be the i...| 5699|AskReddit|
|         ¯\\_(ツ)_/¯| 5673|AskReddit|
|HEY! THAT'S ME!!!...| 5642|    funny|
|Being fucking lat...| 5570|AskReddit|
+--------------------+-----+---------+
only showing top 10 rows



+-------+--------------------+------------------+---------+
|summary|                body|             score|subreddit|
+-------+--------------------+------------------+---------+
|  count|            37868338|          37868338| 37868338|
|   mean|                 NaN| 5.742138960521584| Infinity|
| stddev|                 NaN|48.223446067798875|      NaN|
|    min|How does DNS com...|              -964|  007chan|
|    max|              󾠳󾠶🔞|              6761|     zzzz|
+-------+--------------------+------------------+---------+



In [6]:
from pyspark.sql.functions import lower, col, trim, explode, split, collect_list
import nltk
from nltk.corpus import stopwords

nltk.download("stopwords")
stop_words = set(stopwords.words("english"))

df = dataset.dataframe

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
words = df.select(explode(split(lower(col("body")), " ")).alias("body"))
filtered_words = words.withColumn("body", trim(words.body)).filter(~col("body").isin(stop_words))

In [8]:
from wordcloud import WordCloud
from pyspark.sql.functions import create_map, desc

wordcloud = WordCloud(background_color="white")

In [9]:
filtered_words.cube("body").count().orderBy("body").show()

+--------------------+---------+
|                body|    count|
+--------------------+---------+
|                NULL|389036178|
|                    | 12116731|
|                  ¼|        1|
|                  �|        1|
|          reduction|        1|
|                how|        1|
|                  \b|        1|
|                \b\b|        1|
|           \bmuscled|        1|
|               \bthe|        1|
|              \bwut?|        1|
|                  \t|       26|
|              \t\t\t|        2|
|            \t\t\t\t|        1|
|        \t\t\t\t\t\t|        1|
|  \t\t\t\t\t\t\t\t\t|        1|
|\t\t\t\t\t\t\t\t\...|        1|
|           \t\t\tyou|        1|
|        \t\tpreload:|        1|
|                 \t"|        1|
+--------------------+---------+
only showing top 20 rows

